# Feature Engineering 

## 1. Importing Necessary Libraries

In [1]:
# importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
pd.set_option("display.max_columns",None)
warnings.filterwarnings("ignore")

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
# loading and visualizing the dataset
df = pd.read_csv("C:\\Users\\yozil\\Desktop\\My projects\\11. Video_Game_Sales_Prediction\\data\\raw data\\video_games_sales.csv")
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# shape
df.shape

(16719, 16)

In [4]:
# General information including the memory usage
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [5]:
# column names
df.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')

## 2. Data Cleaning

### 2.1 Dropping Leaky Columns

In [6]:
# first let's mask all leaky variables
leaky_vars = [var for var in df.columns if var.__contains__("Sale") and var not in ["Global_Sales"]]
leaky_vars

['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

In [7]:
# now let's drop the leaky columns
df.drop(leaky_vars, axis = 1, inplace=True)

### 2.2 Data Type Conversion

In [8]:
# let's convert the user score featrue to float data type
df["User_Score"] = np.where(df["User_Score"] == "tbd", np.nan, df["User_Score"])
df["User_Score"] = df["User_Score"].astype("float")

### 2.3 Dropping High Low Cardinality Fetures

In [9]:
# first let's capture all those categorical variables
cat_vars = [var for var in df.select_dtypes("object").columns]
cat_vars

['Name', 'Platform', 'Genre', 'Publisher', 'Developer', 'Rating']

In [10]:
# let's see the unique number of values in each categorical feature
df[cat_vars].nunique()

Name         11562
Platform        31
Genre           12
Publisher      581
Developer     1696
Rating           8
dtype: int64

In [11]:
# let's drop the ["Name", "Publisher", "Developer"] features
high_cardinality_var = ["Name", "Publisher", "Developer"]
df.drop(high_cardinality_var, axis =1, inplace=True)

In [12]:
# let's visualize the dataframe
df.head()

,Platform,Year_of_Release,Genre,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,82.53,76.0,51.0,8.0,322.0,E
1,NES,1985.0,Platform,40.24,NaN,NaN,NaN,NaN,NaN
2,Wii,2008.0,Racing,35.52,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,32.77,80.0,73.0,8.0,192.0,E
4,GB,1996.0,Role-Playing,31.37,NaN,NaN,NaN,NaN,NaN


## 3. Train Test Split

In [13]:
# let's divide our data set in to training and testing sets.
target = "Global_Sales"
x_train, x_test, y_train, y_test = train_test_split(df.drop(target,axis =1),
                                                    df[target],
                                                    test_size=0.1,
                                                    random_state=42)

In [14]:
# shape of each split
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(15047, 8) (15047,) (1672, 8) (1672,)


In [15]:
# let's visualize each split
x_train.head()

,Platform,Year_of_Release,Genre,Critic_Score,Critic_Count,User_Score,User_Count,Rating
14922,PS2,2006.0,Simulation,NaN,NaN,NaN,NaN,NaN
4401,XB,2005.0,Action,50.0,14.0,5.9,21.0,M
11529,NG,1995.0,Fighting,NaN,NaN,NaN,NaN,NaN
11753,GC,2002.0,Racing,55.0,12.0,NaN,NaN,T
4693,PC,2016.0,Shooter,79.0,33.0,5.8,1188.0,M


In [16]:
x_test.head()

,Platform,Year_of_Release,Genre,Critic_Score,Critic_Count,User_Score,User_Count,Rating
6991,XB,2005.0,Action,84.0,47.0,8.0,22.0,T
16195,DS,2010.0,Adventure,NaN,NaN,NaN,NaN,NaN
9862,X360,2016.0,Sports,NaN,NaN,3.4,11.0,E
11152,DS,2005.0,Platform,NaN,NaN,NaN,NaN,NaN
8642,DS,2007.0,Misc,NaN,NaN,NaN,NaN,NaN


In [17]:
y_train.head()

14922    0.03
4401     0.44
11529    0.08
11753    0.08
4693     0.41
Name: Global_Sales, dtype: float64

In [18]:
y_test.head()

6991     0.23
16195    0.01
9862     0.12
11152    0.09
8642     0.16
Name: Global_Sales, dtype: float64

## 4. Feature Engineering

### 4.1 Categorical Variables

In [19]:
# first let's capture all those categorical variables
cat_vars = [var for var in x_train.select_dtypes("object").columns]
cat_vars

['Platform', 'Genre', 'Rating']

#### 4.1.1 Missing Values

In [20]:
# let's see if there are missing values in the categorical features
cat_vars_miss = [var for var in cat_vars if df[var].isnull().sum()>0]
cat_vars_miss

['Genre', 'Rating']

In [21]:
# let's see the percentage of missing values in this records
x_train[cat_vars_miss].isnull().mean()

Genre     0.000133
Rating    0.406061
dtype: float64

1. Genre have missing value < 10% so we will replace missing values here with the mode.
2. Rating have missing values >10% so we will replace missing values with string "Missing".

In [22]:
imp_mode = ["Genre"]
imp_missing = ["Rating"]

In [23]:
# let's impute variables with mode
for var in imp_mode:
    mode = x_train[var].mode()[0]
    print(var, mode)
    x_train[var] = x_train[var].fillna(mode)
    x_test[var] = x_test[var].fillna(mode)

Genre Action


In [24]:
# let's impute the variables with string "Missing"
for var in imp_missing:
    x_train[var] = x_train[var].fillna("Missing")
    x_test[var] = x_test[var].fillna("Missing")
    print("filled Successfully")

filled Successfully


In [25]:
# now let's check the existance of missing values on both the training and testing sets
x_train[cat_vars].isnull().sum()


Platform    0
Genre       0
Rating      0
dtype: int64

In [26]:
# check this for the test set too
x_test[cat_vars].isnull().sum()

Platform    0
Genre       0
Rating      0
dtype: int64

#### 4.1.2 Removing Rare Labels

In [27]:
# first let's find all varaibles with >1% appearance and we will assign the rest with string "Rare"
def frequent_label(data, var, per):
    #let's create a temporary series
    temp = data.groupby(var)[var].count()/len(data[var])
    return temp[temp > per].index

for var in cat_vars:
    frequents = frequent_label(x_train, var, 0.01)
    x_train[var] = np.where(x_train[var].isin(frequents), x_train[var], "Rare")
    x_test[var] = np.where(x_test[var].isin(frequents), x_test[var], "Rare")
    print(var, len(frequents), frequents)
    print()

Platform 18 Index(['3DS', 'DS', 'GBA', 'GC', 'N64', 'PC', 'PS', 'PS2', 'PS3', 'PS4', 'PSP',
       'PSV', 'SAT', 'SNES', 'Wii', 'X360', 'XB', 'XOne'],
      dtype='object', name='Platform')

Genre 12 Index(['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy'],
      dtype='object', name='Genre')

Rating 5 Index(['E', 'E10+', 'M', 'Missing', 'T'], dtype='object', name='Rating')



#### 4.1.3 Encoding Categorical Variables

In [28]:
# let's see the categorical features again
x_train[cat_vars].head()

,Platform,Genre,Rating
14922,PS2,Simulation,Missing
4401,XB,Action,M
11529,Rare,Fighting,Missing
11753,GC,Racing,T
4693,PC,Shooter,M


In [29]:
# let's use the encode categorical variables such that we capture monotonic relationship
def encode_cat(x_train,y_train, x_test,y_test,var,target):
    
    # concatnating the dataset
    train = pd.concat([x_train,pd.DataFrame(y_train)], axis=1)
    # learning encodings from the train set 
    ordered_labels = train.groupby(var)[target].mean().sort_values().index
    ordered_dict = { k:i for i, k in enumerate(ordered_labels)}
    
    # apply encoding to both train and test set
    x_train[var] = x_train[var].map(ordered_dict)
    x_test[var] = x_test[var].map(ordered_dict)
    
#apply the encode cat function
for var in cat_vars:
    encode_cat(x_train,y_train,x_test,y_test,var,"Global_Sales")

In [30]:
# now check for missing values in the data
x_train[cat_vars].isnull().sum()

Platform    0
Genre       0
Rating      0
dtype: int64

In [31]:
# same for the test set
x_test[cat_vars].isnull().sum()

Platform    0
Genre       0
Rating      0
dtype: int64

In [32]:
# visualize the train 
x_train[cat_vars].head()

,Platform,Genre,Rating
14922,9,3,0
4401,4,5,5
11529,18,6,0
11753,6,8,3
4693,3,10,5


In [33]:
# visualize the test set data frame
x_test[cat_vars].head()

,Platform,Genre,Rating
6991,4,5,3
16195,5,0,0
9862,15,7,4
11152,5,11,0
8642,5,4,0


### 4.2 Numerical Variables

In [34]:
x_train.head()

,Platform,Year_of_Release,Genre,Critic_Score,Critic_Count,User_Score,User_Count,Rating
14922,9,2006.0,3,NaN,NaN,NaN,NaN,0
4401,4,2005.0,5,50.0,14.0,5.9,21.0,5
11529,18,1995.0,6,NaN,NaN,NaN,NaN,0
11753,6,2002.0,8,55.0,12.0,NaN,NaN,3
4693,3,2016.0,10,79.0,33.0,5.8,1188.0,5


In [35]:
# first let's mask numerical columns
num_vars = [var for var in x_train.columns if var not in cat_vars]
num_vars

['Year_of_Release', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count']

#### 4.2.1 Temporal Variables

In [36]:
# let's mask out the temporal variables
temp_vars = [var for var in num_vars if var.__contains__("Year")]
temp_vars

['Year_of_Release']

##### 4.2.1.1 Temporal Missing Values

In [37]:
# first let's see if there is a missing values in the temporal variables
x_train[temp_vars].isnull().any()

Year_of_Release    True
dtype: bool

In [38]:
# there are missing values in the temporal variables , let's see the percentage of missing values
x_train[temp_vars].isnull().mean()

Year_of_Release    0.015884
dtype: float64

In [39]:
# the missing value is less than 2% so we can impute the mode value to handle this missing values 
for var in temp_vars:
    mode = x_train[var].mode()[0]
    x_train[var] = x_train[var].fillna(mode)
    x_test[var] = x_test[var].fillna(mode)

In [40]:
# now let's check the existance of null values in the temporal features
x_train[temp_vars].isnull().sum()

Year_of_Release    0
dtype: int64

In [41]:
# now let's check the existance of null values in the temporal features
x_test[temp_vars].isnull().sum()

Year_of_Release    0
dtype: int64

##### 4.2.1.2 Extract the Age Information

In [42]:
# let's extract the Age information and overwrite the year
today = datetime.today()
year = today.year
year

for var in temp_vars:
    x_train["Age"] = year - x_train[var]
    x_test["Age"] = year - x_test[var]

In [43]:
# now let's drop the release year feature
x_train.drop("Year_of_Release", axis =1, inplace=True)
x_test.drop("Year_of_Release", axis =1, inplace=True)

In [44]:
x_train.head()

,Platform,Genre,Critic_Score,Critic_Count,User_Score,User_Count,Rating,Age
14922,9,3,NaN,NaN,NaN,NaN,0,18.0
4401,4,5,50.0,14.0,5.9,21.0,5,19.0
11529,18,6,NaN,NaN,NaN,NaN,0,29.0
11753,6,8,55.0,12.0,NaN,NaN,3,22.0
4693,3,10,79.0,33.0,5.8,1188.0,5,8.0


In [45]:
# now let's visualize the numerical features dataframe
num_vars = [var for var in num_vars +["Age"] if var not in ["Year_of_Release"]]
x_train[num_vars].head()

,Critic_Score,Critic_Count,User_Score,User_Count,Age
14922,NaN,NaN,NaN,NaN,18.0
4401,50.0,14.0,5.9,21.0,19.0
11529,NaN,NaN,NaN,NaN,29.0
11753,55.0,12.0,NaN,NaN,22.0
4693,79.0,33.0,5.8,1188.0,8.0


#### 4.2.1 Discrete Numerical Variables

In [46]:
# let's mask the discrete numerical varaibles
disc_vars = [var for var in num_vars if x_train[var].nunique() < 20]
disc_vars


[]

As we see we don't have any Discrete variables

#### 4.2.2 Continuous Numerical Varaibles

In [47]:
# first let's mask the continuous numerical variables
cont_vars = [var for var in num_vars if var not in ["Age"]]
cont_vars

['Critic_Score', 'Critic_Count', 'User_Score', 'User_Count']

##### 4.2.2.1 Missing Values

In [48]:
# first let's see the percentage of missing values in each feature
x_train[cont_vars].isnull().mean()

Critic_Score    0.515186
Critic_Count    0.515186
User_Score      0.548415
User_Count      0.548415
dtype: float64

All the features consists >50% of missing values so we drop all of them

In [49]:
# let's drop all the continous variables columns
x_train.drop(cont_vars, axis =1, inplace= True)
x_test.drop(cont_vars, axis =1, inplace= True)

In [50]:
# now let's visualize the train and test dataframe
x_train.head()

,Platform,Genre,Rating,Age
14922,9,3,0,18.0
4401,4,5,5,19.0
11529,18,6,0,29.0
11753,6,8,3,22.0
4693,3,10,5,8.0


In [51]:
x_test.head()

,Platform,Genre,Rating,Age
6991,4,5,3,19.0
16195,5,0,0,14.0
9862,15,7,4,8.0
11152,5,11,0,19.0
8642,5,4,0,17.0


### 4.3 Feature Scaling

In [52]:
# let's apply standard scalling to the train and test set of the data
# first let's initialize the scaler
scaler = StandardScaler()

In [53]:
# now let's fit the scaler with the trianing set
scaler.fit(x_train)

StandardScaler()

In [54]:
# now let's transform both the training and testing sets
x_train = pd.DataFrame(scaler.transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns = x_test.columns)

In [55]:
# now let's visualize the test set
x_test.head()

,Platform,Genre,Rating,Age
0,-0.975676,-0.248851,0.467379,0.259447
1,-0.764375,-1.889000,-1.120366,-0.597159
2,1.348638,0.407209,0.996627,-1.625085
3,-0.764375,1.719328,-1.120366,0.259447
4,-0.764375,-0.576881,-1.120366,-0.083196


In [56]:
# now let's visualize the tiarning set
x_train.head()

,Platform,Genre,Rating,Age
0,0.080830,-0.904911,-1.120366,0.088126
1,-0.975676,-0.248851,1.525876,0.259447
2,1.982542,0.079179,-1.120366,1.972658
3,-0.553074,0.735239,0.467379,0.773410
4,-1.186978,1.391298,1.525876,-1.625085


### 4.3.1 Saving The Scaler

In [57]:
joblib.dump(scaler, "C:\\Users\\yozil\\Desktop\\My projects\\11. Video_Game_Sales_Prediction\\models\\scaler.joblib")

['C:\\Users\\yozil\\Desktop\\My projects\\11. Video_Game_Sales_Prediction\\models\\scaler.joblib']

## 5. Saving The DataFrames

In [58]:
x_train.to_csv("C:\\Users\\yozil\\Desktop\\My projects\\11. Video_Game_Sales_Prediction\\data\\processed data\\x_train.csv", index = False)
x_test.to_csv("C:\\Users\\yozil\\Desktop\\My projects\\11. Video_Game_Sales_Prediction\\data\\processed data\\x_test.csv",index = False)
y_train.to_csv("C:\\Users\\yozil\\Desktop\\My projects\\11. Video_Game_Sales_Prediction\\data\\processed data\\y_train.csv",index = False)
y_test.to_csv("C:\\Users\\yozil\\Desktop\\My projects\\11. Video_Game_Sales_Prediction\\data\\processed data\\y_test.csv",index = False)